This notebook is for development. It contains easier to follow, but very slow Python code. It is too slow, so in other notebooks, used for computations, I rewrote this with Numba.

In [16]:
import numpy as np

# Prepare the grid.
MAX_N=26
K = 2*(MAX_N//2)
box_size = 2*K+1
dx,dy,dz=1,box_size,box_size**2
CENTER_COORD = K*(dx+dy+dz)

# Pre-calculate geometry.
CUBE = [[1,3,4,2],[0,2,5,3],[0,4,5,1],[0,1,5,4],[0,3,5,2],[1,2,4,3]]
DELTAS = np.array([dy,dz,dx,-dx,-dz,-dy]) # "+y","+z","+x","-x","-z","-y"
WEDGE_ID_TO_FACE_IDS = dict()
FACE_IDS_TO_WEDGE_ID = dict()

def register_wedge(f1, f2, wedge_id):
  WEDGE_ID_TO_FACE_IDS[wedge_id] = (f1, f2)
  FACE_IDS_TO_WEDGE_ID[(f1, f2)] = wedge_id

for i, (f1, f2) in enumerate([(0,1),(0,2),(0,3),(0,4),(1,2),(1,3)]):
  register_wedge(f1,f2, i+1)
  register_wedge(f2,f1, i+1+16)
  register_wedge(5-f1,5-f2, 13-(i+1))
  register_wedge(5-f2,5-f1, 13-(i+1)+16)
  
WEDGE_ID_TO_NEXT_DELTA = np.zeros(36, dtype=np.int64)
ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID = np.zeros(36*4, dtype=np.int64)
for f1 in range(6):
  for f2 in CUBE[f1]:
    wedge_id = FACE_IDS_TO_WEDGE_ID[(f1,f2)]
    f1p=5-f2
    f2p=[5-f1,0,f1,0]
    f2p[1]=CUBE[f1][(CUBE[f1].index(f2)+1)%4]
    f2p[3]=5-f2p[1]
    WEDGE_ID_TO_NEXT_DELTA[wedge_id] = DELTAS[f1p]
    for rot in range(4):
      ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID[wedge_id+rot*36] = FACE_IDS_TO_WEDGE_ID[(f1p, f2p[rot])]

class Wedge:
  def __init__(self, coord, wedge_id):
    self.coord = coord
    self.wedge_id = wedge_id
  
  def __repr__(self):
    x = self.coord%box_size
    y = (self.coord//box_size)%box_size
    z = (self.coord//box_size)//box_size
    f1,f2 = WEDGE_ID_TO_FACE_IDS[self.wedge_id]
    return f"({x},{y},{z};{f1}->{f2})"

  def get_next(self, rot):
    return Wedge(self.coord + WEDGE_ID_TO_NEXT_DELTA[self.wedge_id],
                 ROT_AND_WEDGE_ID_TO_NEXT_WEDGE_ID[self.wedge_id+36*rot])
  
  def occ_type(self):
    return self.wedge_id & 15

def is_shape_valid(formula):
  wedges=[Wedge(CENTER_COORD,FACE_IDS_TO_WEDGE_ID[(0,3)])]
  cubes = {wedges[0].coord: wedges[0].occ_type()}
  assert len(formula)+1 <= MAX_N
  for rot in formula:
    next_wedge = wedges[-1].get_next(rot)
    if next_wedge.coord not in cubes:
      cubes[next_wedge.coord] = next_wedge.occ_type()
    else:
      assert cubes[next_wedge.coord] > 0
      can_push = (cubes[next_wedge.coord]+next_wedge.occ_type() == 13)
      if not can_push:
        return False
      cubes[next_wedge.coord]+=next_wedge.occ_type()
      assert cubes[next_wedge.coord]==13
    wedges.append(next_wedge)
  return True

def enumerate_valid_shapes(n):
  ans = []
  rots = np.zeros(n-1, dtype=np.int64)
  for i in range(4**(n-1)):
    for j in range(n-1):
      rots[j]=(i>>(2*j))&3
    if is_shape_valid(rots): 
      ans.append("".join(map(str, rots)))
  return ans

for n in range(1,13):
  valid_shapes = enumerate_valid_shapes(n)
  num_shapes = len(valid_shapes)
  num_palindromes = sum(1 for s in valid_shapes if s[::-1]==s)
  num_shapes_up_to_reverse = len(set(min(s, s[::-1]) for s in valid_shapes))
  assert 2*num_shapes_up_to_reverse == num_palindromes + num_shapes
  print("n=%d. %d shapes. %d up to reverse. %d palindromes." % (
    n, num_shapes, num_shapes_up_to_reverse, num_palindromes))

n=1. 1 shapes. 1 up to reverse. 1 palindromes.
n=2. 4 shapes. 4 up to reverse. 4 palindromes.
n=3. 16 shapes. 10 up to reverse. 4 palindromes.
n=4. 64 shapes. 40 up to reverse. 16 palindromes.
n=5. 241 shapes. 127 up to reverse. 13 palindromes.
n=6. 920 shapes. 490 up to reverse. 60 palindromes.
n=7. 3384 shapes. 1718 up to reverse. 52 palindromes.
n=8. 12585 shapes. 6403 up to reverse. 221 palindromes.
n=9. 46471 shapes. 23328 up to reverse. 185 palindromes.
n=10. 172226 shapes. 86514 up to reverse. 802 palindromes.
n=11. 633138 shapes. 316919 up to reverse. 700 palindromes.
n=12. 2333757 shapes. 1168357 up to reverse. 2957 palindromes.
